In [ ]:
from agential.prompting.cot.prompting import CoT

from agential.prompting.cot.prompts import (
    COT_INSTRUCTION_AMBIGNQ, 
    COT_INSTRUCTION_FEVER, 
    COT_INSTRUCTION_GSM8K, 
    COT_INSTRUCTION_HOTPOTQA, 
    COT_INSTRUCTION_SVAMP, 
    COT_INSTRUCTION_TRIVIAQA,
    COT_INSTRUCTION_TABMWP,
    COT_INSTRUCTION_HUMANEVAL,
    COT_INSTRUCTION_MBPP,
)
from agential.core.fewshots.ambignq import AMBIGNQ_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.fever import FEVER_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.gsm8k import GSM8K_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.svamp import SVAMP_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.triviaqa import TRIVIAQA_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.tabmwp import TABMWP_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.humaneval import HUMANEVAL_FEWSHOT_EXAMPLES_COT
from agential.core.fewshots.mbpp import MBPP_FEWSHOT_EXAMPLES_COT

import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()

from agential.core.llm import LLM

llm = LLM("gpt-3.5-turbo")

In [1]:
import json

# Open and read the JSON file
with open('../data/hotpot_dev_v1_simplified.json', 'r') as file:
    data = json.load(file)

# Print the data
print(data[:5])

[{'question': 'Were Scott Derrickson and Ed Wood of the same nationality?', 'answer': 'yes', 'type': 'comparison'}, {'question': 'What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?', 'answer': 'Chief of Protocol', 'type': 'bridge'}, {'question': 'What science fantasy young adult series, told in first person, has a set of companion books narrating the stories of enslaved worlds and alien species?', 'answer': 'Animorphs', 'type': 'bridge'}, {'question': 'Are the Laleli Mosque and Esma Sultan Mansion located in the same neighborhood?', 'answer': 'no', 'type': 'comparison'}, {'question': 'The director of the romantic comedy "Big Stone Gap" is based in what New York city?', 'answer': 'Greenwich Village, New York City', 'type': 'bridge'}]


In [ ]:
from agential.eval.metrics.classification import EM


method = CoT(
    llm=llm,
    benchmark="hotpotqa",
)

samples = 20
num_correct = 0

for i in data[:samples]:

    question = i["question"]
    answer = i["answer"]

    out = method.generate(
        question=question,
        key=answer,
        examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT, 
        prompt=COT_INSTRUCTION_HOTPOTQA,
        additional_keys={},
        num_retries=1,
        warming=[None],
    )
    is_correct = EM(out.answer, answer)
    print(question)
    print(answer, "\t\t", out.answer, "\t\t", is_correct, end="\n\n")

    num_correct += int(is_correct)

    precision_out = precision(out.answer , answer , True)

    recall_out = recall(out.answer , answer , True)

    f1_out = f1(out.answer , answer , True)
    
print(f"{num_correct}/{samples}")